# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [8]:
# Step 1: Create a data frame that contains the total quantity of each product purchased by each customer
grouped_data = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'}).reset_index()
print(grouped_data.head())

   CustomerID                    ProductName  Quantity
0          33               Apricots - Dried         1
1          33              Assorted Desserts         1
2          33        Bandage - Flexible Neon         1
3          33  Bar Mix - Pina Colada, 355 Ml         1
4          33         Beans - Kidney, Canned         1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [10]:
# Step 2: Create a product by customer matrix
matrix = pd.pivot_table(data=grouped_data, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0)
print(matrix)

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube              0      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   
Appetizer - Sausage Rolls              0      0      0      0      0      0   
Apricots - Dried                       1      0      0      0      1      0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                   0      0      0      0      0      0   
Yoghurt Tubes                          0      0      0      0      0      0   
Yogurt - Blueberry, 175 Gr             0      1      0      0      0      0   
Yogurt - French Vanilla                1      0      0      1      0      0   
Zucchini - Yellow                      0      0     

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
# Step 3: Create a customer similarity matrix
customer_similarity = pdist(matrix.values.T, metric='euclidean')
customer_similarity_matrix = pd.DataFrame(squareform(customer_similarity), index=matrix.columns, columns=matrix.columns)

# Print the customer similarity matrix
print(customer_similarity_matrix)

CustomerID       33          200         264         356         412    \
CustomerID                                                               
33            0.000000   11.916375   10.488088   11.224972   11.401754   
200          11.916375    0.000000   11.747340   12.083046   12.569805   
264          10.488088   11.747340    0.000000   11.489125   11.224972   
356          11.224972   12.083046   11.489125    0.000000   12.083046   
412          11.401754   12.569805   11.224972   12.083046    0.000000   
...                ...         ...         ...         ...         ...   
97928       220.501701  220.202180  219.136943  219.952268  219.733930   
98069       217.188858  215.728997  216.612557  217.437347  217.446545   
98159       228.628520  228.010965  228.081126  228.098663  227.997807   
98185       239.000000  239.037654  238.266657  238.493186  238.396728   
98200       229.773802  229.704158  229.773802  229.464594  228.927936   

CustomerID       464         477     

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [14]:
# Step 4: Generate a list of the top 5 most similar customers for a specific CustomerID
specific_customer_id = 33  # Replace 123 with the desired CustomerID

similar_customers = customer_similarity_matrix[specific_customer_id].nlargest(6)[1:]
similar_customer_ids = similar_customers.index.tolist()

# Print the list of top 5 most similar customers
print("Top 5 most similar customers for Customer", specific_customer_id)
print(similar_customer_ids)


Top 5 most similar customers for Customer 33
[97324, 95017, 92492, 91777, 92525]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [15]:
# Step 5: Select records for the list of similar CustomerIDs
similar_customer_records = grouped_data[grouped_data['CustomerID'].isin(similar_customer_ids)]

# Print the selected records
print(similar_customer_records)

       CustomerID                        ProductName  Quantity
59478       91777  Appetizer - Mini Egg Roll, Shrimp        24
59479       91777                    Banana - Leaves        24
59480       91777                     Banana Turning        24
59481       91777              Bandage - Fexible 1x3        24
59482       91777            Bandage - Flexible Neon        24
...           ...                                ...       ...
62914       97324            Wine - Redchard Merritt        25
62915       97324  Wine - Vineland Estate Semi - Dry        25
62916       97324        Wine - White, Colubia Cresh        25
62917       97324                    Wonton Wrappers        25
62918       97324            Yogurt - French Vanilla        25

[373 rows x 3 columns]


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [16]:
# Step 6: Aggregate customer purchase records by ProductName and rank them by quantity
similar_customers_purchases = similar_customer_records.groupby('ProductName').agg({'Quantity': 'sum'}).reset_index()
ranked_purchases = similar_customers_purchases.sort_values('Quantity', ascending=False)

# Print the ranked purchases
print("Ranked purchases by the 5 most similar customers:")
print(ranked_purchases)

Ranked purchases by the 5 most similar customers:
                          ProductName  Quantity
228           Turnip - White, Organic       146
22      Beef - Tenderlion, Center Cut       122
243      Whmis - Spray Bottle Trigger        98
162       Pastry - Butterscotch Baked        98
66                        Clam Nectar        98
..                                ...       ...
113         Ice Cream Bar - Oreo Cone        24
111         Ice Cream Bar - Drumstick        24
110        Hot Chocolate - Individual        24
107                Halibut - Fletches        24
132  Longos - Grilled Salmon With Bbq        24

[265 rows x 2 columns]


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [20]:
# Step 7: Filter the ranked products for products not yet purchased by the chosen customer and recommend the top 5 products
customer_purchases = matrix.loc[:, [specific_customer_id]].reset_index()
customer_purchases.columns = ['ProductName', specific_customer_id]

recommended_products = ranked_purchases.merge(customer_purchases, on='ProductName', how='left')
recommended_products = recommended_products[recommended_products[specific_customer_id].isnull()]
recommended_products = recommended_products.nlargest(5, 'Quantity')

# Print the top 5 recommended products
print("Top 5 recommended products for Customer", specific_customer_id)
print(recommended_products[['ProductName', 'Quantity']])

Top 5 recommended products for Customer 33
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [22]:
# Step 8: Generate product recommendations for all customers
recommendations_dict = {}  # Create an empty dictionary to hold recommendations for all customers

customer_ids = matrix.columns.tolist()  # Create a list of unique CustomerIDs

for customer_id in customer_ids:
    similar_customers = customer_similarity_matrix[customer_id].nlargest(6)[1:]
    similar_customer_ids = similar_customers.index.tolist()
    
    similar_customer_records = grouped_data[grouped_data['CustomerID'].isin(similar_customer_ids)]
    
    similar_customers_purchases = similar_customer_records.groupby('ProductName').agg({'Quantity': 'sum'}).reset_index()
    ranked_purchases = similar_customers_purchases.sort_values('Quantity', ascending=False)
    
    customer_purchases = matrix.loc[:, [customer_id]].reset_index()
    customer_purchases.columns = ['ProductName', customer_id]
    
    recommended_products = ranked_purchases.merge(customer_purchases, on='ProductName', how='left')
    recommended_products = recommended_products[recommended_products[customer_id].isnull()]
    recommended_products = recommended_products.nlargest(5, 'Quantity')
    
    recommendations_dict[customer_id] = recommended_products[['ProductName', 'Quantity']]

# Print the recommendations for all customers
for customer_id, recommendations in recommendations_dict.items():
    print("Top 5 recommended products for Customer", customer_id)
    print(recommendations)
    print()

Top 5 recommended products for Customer 33
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 200
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 264
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 356
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 412
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 464
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 477
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 639
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 649
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 669
Empty DataFrame
Colum

Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 81813
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 81845
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 81876
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 81981
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 82057
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 82146
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 82161
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 82432
Empty DataFrame
Columns: [ProductName, Quantity]
Index: []

Top 5 recommended products for Customer 82440
Empty DataFrame
Columns: [ProductName, Quantit

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [28]:
# Step 9: Store the recommendations in a Pandas DataFrame
recommendations_df = pd.DataFrame.from_dict(recommendations_dict, orient='index').reset_index()
recommendations_df.columns = ['CustomerID'] + [f'Recommendation{i+1}' for i in range(5)]

# Print the recommendations DataFrame
print(recommendations_df)

ValueError: Must pass 2-d input. shape=(1000, 0, 2)

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [29]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Step 1: Load the data
data_df = pd.read_csv('data.csv')

# Step 2: Pivot the data
pivot_df = data_df.pivot(index='CustomerID', columns='ProductName', values='Quantity').fillna(0)

# Step 3: Calculate the pairwise distances
distances = pdist(pivot_df.values.T, metric='correlation')

# Step 4: Convert distances to a square matrix
dist_matrix = squareform(distances)

# Step 5: Create a dictionary to store recommendations
recommendations_dict = {}

# Step 6: Generate recommendations for each customer
for i, customer_id in enumerate(pivot_df.index):
    customer_row = dist_matrix[i]
    similar_customers = pivot_df.index[customer_row.argsort()[1:6]]
    recommendations_dict[customer_id] = list(similar_customers)

# Step 7: Store the recommendations in a Pandas DataFrame
recommendations_df = pd.DataFrame.from_dict(recommendations_dict, orient='index').reset_index()
recommendations_df.columns = ['CustomerID'] + [f'Recommendation{i+1}' for i in range(5)]

# Print the recommendations DataFrame
print(recommendations_df)

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'